Начнем с предобработки текста. Данные взяты с Kaggle, это набор статей с медиума: https://www.kaggle.com/datasets/fabiochiusano/medium-articles

In [34]:
import pandas as pd
from collections import Counter
import numpy as np

In [2]:
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk import wordnet, pos_tag
from nltk import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import metrics
from nltk.corpus import wordnet
import string
from nltk import pos_tag
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [3]:
from nltk.corpus import stopwords

In [4]:
import re

In [5]:
import gensim
from gensim.parsing.preprocessing import remove_stopwords
from gensim import corpora

In [6]:
from multiprocess import Pool

In [7]:
df = pd.read_csv('/content/drive/MyDrive/forcolab/archive (4).zip')

In [8]:
df

,title,text,url,authors,timestamp,tags
0,Mental Note Vol. 24,Photo by Josh Riemer on Unsplash\n\nMerry Chri...,https://medium.com/invisible-illness/mental-no...,['Ryan Fan'],2020-12-26 03:38:10.479000+00:00,"['Mental Health', 'Health', 'Psychology', 'Sci..."
1,Your Brain On Coronavirus,Your Brain On Coronavirus\n\nA guide to the cu...,https://medium.com/age-of-awareness/how-the-pa...,['Simon Spichak'],2020-09-23 22:10:17.126000+00:00,"['Mental Health', 'Coronavirus', 'Science', 'P..."
2,Mind Your Nose,Mind Your Nose\n\nHow smell training can chang...,https://medium.com/neodotlife/mind-your-nose-f...,[],2020-10-10 20:17:37.132000+00:00,"['Biotechnology', 'Neuroscience', 'Brain', 'We..."
3,The 4 Purposes of Dreams,Passionate about the synergy between science a...,https://medium.com/science-for-real/the-4-purp...,['Eshan Samaranayake'],2020-12-21 16:05:19.524000+00:00,"['Health', 'Neuroscience', 'Mental Health', 'P..."
4,Surviving a Rod Through the Head,"You’ve heard of him, haven’t you? Phineas Gage...",https://medium.com/live-your-life-on-purpose/s...,['Rishav Sinha'],2020-02-26 00:01:01.576000+00:00,"['Brain', 'Health', 'Development', 'Psychology..."
...,...,...,...,...,...,...
192363,Why do you need a cleaning service?,What could be more important than having a tid...,https://medium.com/@ozneedcleaningau/why-do-yo...,[],2021-11-16 08:17:08.950000+00:00,"['Cleaning', 'Cleaning Services', 'Cleaning Co..."
192364,Daily cleaning and maintenance of bedding,Daily cleaning and maintenance of bedding\n\nW...,https://medium.com/@a198blwt/daily-cleaning-an...,[],2021-11-16 05:27:05.359000+00:00,"['Bedding', 'Cleaning', 'Maintain']"
192365,Beneficial Advice on Bond Cleaning!,The most important chore at the end is bond cl...,https://medium.com/@princegohil/beneficial-adv...,['Prince Shrawan'],2021-11-26 08:20:27.660000+00:00,"['Cleaning', 'End Of Lease Cleaning', 'Cleaners']"
192366,How I Learned Romanian in 37 Easy Steps,How I Learned Romanian in 37 Easy Steps\n\nHey...,https://medium.com/@lifeinromania/how-i-learne...,['Sam Ursu'],2017-11-27 08:09:19.025000+00:00,"['Romania', 'Language Learning', 'Storyofmylife']"


In [9]:
df.columns

Index(['title', 'text', 'url', 'authors', 'timestamp', 'tags'], dtype='object')

Дропнем колонку со временем:

In [10]:
df_without_time = df.drop('timestamp', axis=1)

In [11]:
df_without_time

,title,text,url,authors,tags
0,Mental Note Vol. 24,Photo by Josh Riemer on Unsplash\n\nMerry Chri...,https://medium.com/invisible-illness/mental-no...,['Ryan Fan'],"['Mental Health', 'Health', 'Psychology', 'Sci..."
1,Your Brain On Coronavirus,Your Brain On Coronavirus\n\nA guide to the cu...,https://medium.com/age-of-awareness/how-the-pa...,['Simon Spichak'],"['Mental Health', 'Coronavirus', 'Science', 'P..."
2,Mind Your Nose,Mind Your Nose\n\nHow smell training can chang...,https://medium.com/neodotlife/mind-your-nose-f...,[],"['Biotechnology', 'Neuroscience', 'Brain', 'We..."
3,The 4 Purposes of Dreams,Passionate about the synergy between science a...,https://medium.com/science-for-real/the-4-purp...,['Eshan Samaranayake'],"['Health', 'Neuroscience', 'Mental Health', 'P..."
4,Surviving a Rod Through the Head,"You’ve heard of him, haven’t you? Phineas Gage...",https://medium.com/live-your-life-on-purpose/s...,['Rishav Sinha'],"['Brain', 'Health', 'Development', 'Psychology..."
...,...,...,...,...,...
192363,Why do you need a cleaning service?,What could be more important than having a tid...,https://medium.com/@ozneedcleaningau/why-do-yo...,[],"['Cleaning', 'Cleaning Services', 'Cleaning Co..."
192364,Daily cleaning and maintenance of bedding,Daily cleaning and maintenance of bedding\n\nW...,https://medium.com/@a198blwt/daily-cleaning-an...,[],"['Bedding', 'Cleaning', 'Maintain']"
192365,Beneficial Advice on Bond Cleaning!,The most important chore at the end is bond cl...,https://medium.com/@princegohil/beneficial-adv...,['Prince Shrawan'],"['Cleaning', 'End Of Lease Cleaning', 'Cleaners']"
192366,How I Learned Romanian in 37 Easy Steps,How I Learned Romanian in 37 Easy Steps\n\nHey...,https://medium.com/@lifeinromania/how-i-learne...,['Sam Ursu'],"['Romania', 'Language Learning', 'Storyofmylife']"


In [12]:
df_without_time.shape

(192368, 5)

Так как датасет довольно объемный, то есть смысл распараллелить предобработку. Использовать буду стэмминг.

In [13]:
import string

In [14]:
PUNCTUATION = re.compile('[%s]' % re.escape(string.punctuation))

In [15]:
stemmer = SnowballStemmer(language='english')
stops = set(stopwords.words("english")) 
def tokenize(data):
    #print(data.split())
    return data.split()

def lowercase_filter(tokens):
    return [token.lower() for token in tokens]

def punctuation_filter(tokens):
    return [PUNCTUATION.sub('', token) for token in tokens]

def stopword_filter(tokens):
    return [token for token in tokens if token not in stops]

def stem_filter(tokens):
    return [stemmer.stem(word) for word in tokens]

In [16]:
def analyze(text):
    tokens = tokenize(text)
    tokens = lowercase_filter(tokens)
    tokens = punctuation_filter(tokens)
    tokens = stopword_filter(tokens)
    tokens = stem_filter(tokens)

    return [token for token in tokens if token]

In [17]:
def analyze_pool(data):
  with Pool(20) as p:
    tokens = list(p.imap(analyze, data))
  return tokens

In [19]:
#!pip install pyblaze

In [20]:
#import pyblaze.multiprocessing as xmp

In [21]:
#def parallel(texts):
#    tokenizer = xmp.Vectorizer(tokenize, num_workers=20)
#    return tokenizer.process(texts)

Будем работать с частью датасета из-за приближающегося дедлайна и нехватки времени). 10000 статей обрабатываются около 1.5 минуты с распараллеливанием, весь датасет около 40 минут. Обрабатывать будем заголовок статьи + текст, т.к. из заголовка можно извлечь слишком мало информации.

In [27]:
df_part = df_without_time.loc[1:10000]

In [28]:
#df_part['all_text'] =  df_part['title'] + ' ' + df_part['text']

In [29]:
df_part

,title,text,url,authors,tags
1,Your Brain On Coronavirus,Your Brain On Coronavirus\n\nA guide to the cu...,https://medium.com/age-of-awareness/how-the-pa...,['Simon Spichak'],"['Mental Health', 'Coronavirus', 'Science', 'P..."
2,Mind Your Nose,Mind Your Nose\n\nHow smell training can chang...,https://medium.com/neodotlife/mind-your-nose-f...,[],"['Biotechnology', 'Neuroscience', 'Brain', 'We..."
3,The 4 Purposes of Dreams,Passionate about the synergy between science a...,https://medium.com/science-for-real/the-4-purp...,['Eshan Samaranayake'],"['Health', 'Neuroscience', 'Mental Health', 'P..."
4,Surviving a Rod Through the Head,"You’ve heard of him, haven’t you? Phineas Gage...",https://medium.com/live-your-life-on-purpose/s...,['Rishav Sinha'],"['Brain', 'Health', 'Development', 'Psychology..."
5,"Mentally, Young Adults Are Suffering Most From...","Mentally, Young Adults Are Suffering Most From...",https://medium.com/the-partnered-pen/mentally-...,['Ryan Fan'],"['Society', 'Mental Health', 'Health', 'Nonfic..."
...,...,...,...,...,...
9996,Adding a Border to the Image using NumPy,"In this article, we will learn how to add/draw...",https://medium.com/analytics-vidhya/adding-a-b...,['Sameeruddin Mohammed'],"['Numpy', 'Python Programming', 'Matrix Operat..."
9997,Connecting with Your Ancestors,Every person is a quotation from all their anc...,https://medium.com/grace-and-magic/connecting-...,['Katherine Grace'],"['Halloween', 'Self-awareness', 'Journaling', ..."
9998,Learn how to create beautiful and insightful c...,The Pretty: Advanced plotting with Seaborn\n\n...,https://towardsdatascience.com/plotting-with-p...,['Fabian Bosler'],"['Data Visualization', 'Data Analysis', 'Data ..."
9999,Five New Books to Get Your Hands On in 2020,Five New Books to Get Your Hands On in 2020\n\...,https://karintaglang.medium.com/five-new-books...,['Karin A. R. Taglang'],"['Books', 'Literature', 'Reading', 'Fiction', ..."


In [30]:
df_part['clean_text'] = analyze_pool(df_part['text'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [32]:
df_part

,title,text,url,authors,tags,clean_text
1,Your Brain On Coronavirus,Your Brain On Coronavirus\n\nA guide to the cu...,https://medium.com/age-of-awareness/how-the-pa...,['Simon Spichak'],"['Mental Health', 'Coronavirus', 'Science', 'P...","[brain, coronavirus, guid, curious, troubl, im..."
2,Mind Your Nose,Mind Your Nose\n\nHow smell training can chang...,https://medium.com/neodotlife/mind-your-nose-f...,[],"['Biotechnology', 'Neuroscience', 'Brain', 'We...","[mind, nose, smell, train, chang, brain, six, ..."
3,The 4 Purposes of Dreams,Passionate about the synergy between science a...,https://medium.com/science-for-real/the-4-purp...,['Eshan Samaranayake'],"['Health', 'Neuroscience', 'Mental Health', 'P...","[passion, synergi, scienc, technolog, provid, ..."
4,Surviving a Rod Through the Head,"You’ve heard of him, haven’t you? Phineas Gage...",https://medium.com/live-your-life-on-purpose/s...,['Rishav Sinha'],"['Brain', 'Health', 'Development', 'Psychology...","[you'v, heard, haven't, phinea, gage, railroad..."
5,"Mentally, Young Adults Are Suffering Most From...","Mentally, Young Adults Are Suffering Most From...",https://medium.com/the-partnered-pen/mentally-...,['Ryan Fan'],"['Society', 'Mental Health', 'Health', 'Nonfic...","[mental, young, adult, suffer, covid, “when, c..."
...,...,...,...,...,...,...
9996,Adding a Border to the Image using NumPy,"In this article, we will learn how to add/draw...",https://medium.com/analytics-vidhya/adding-a-b...,['Sameeruddin Mohammed'],"['Numpy', 'Python Programming', 'Matrix Operat...","[articl, learn, adddraw, border, imag, use, op..."
9997,Connecting with Your Ancestors,Every person is a quotation from all their anc...,https://medium.com/grace-and-magic/connecting-...,['Katherine Grace'],"['Halloween', 'Self-awareness', 'Journaling', ...","[everi, person, quotat, ancestor, —, ralph, wa..."
9998,Learn how to create beautiful and insightful c...,The Pretty: Advanced plotting with Seaborn\n\n...,https://towardsdatascience.com/plotting-with-p...,['Fabian Bosler'],"['Data Visualization', 'Data Analysis', 'Data ...","[pretti, advanc, plot, seaborn, photo, pavel, ..."
9999,Five New Books to Get Your Hands On in 2020,Five New Books to Get Your Hands On in 2020\n\...,https://karintaglang.medium.com/five-new-books...,['Karin A. R. Taglang'],"['Books', 'Literature', 'Reading', 'Fiction', ...","[five, new, book, get, hand, 2020, ya, genr, i..."


Код для функций ранжирования и индексирования также прикрепляю здесь:

Начнем с индексирования. 

Попробуем реализовать обратный индекс. Начнем с того, что добавим индексы к документам(просто по порядку).

In [35]:
df_part['ID'] = np.arange(len(df_part))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [36]:
df_part

,title,text,url,authors,tags,clean_text,ID
1,Your Brain On Coronavirus,Your Brain On Coronavirus\n\nA guide to the cu...,https://medium.com/age-of-awareness/how-the-pa...,['Simon Spichak'],"['Mental Health', 'Coronavirus', 'Science', 'P...","[brain, coronavirus, guid, curious, troubl, im...",0
2,Mind Your Nose,Mind Your Nose\n\nHow smell training can chang...,https://medium.com/neodotlife/mind-your-nose-f...,[],"['Biotechnology', 'Neuroscience', 'Brain', 'We...","[mind, nose, smell, train, chang, brain, six, ...",1
3,The 4 Purposes of Dreams,Passionate about the synergy between science a...,https://medium.com/science-for-real/the-4-purp...,['Eshan Samaranayake'],"['Health', 'Neuroscience', 'Mental Health', 'P...","[passion, synergi, scienc, technolog, provid, ...",2
4,Surviving a Rod Through the Head,"You’ve heard of him, haven’t you? Phineas Gage...",https://medium.com/live-your-life-on-purpose/s...,['Rishav Sinha'],"['Brain', 'Health', 'Development', 'Psychology...","[you'v, heard, haven't, phinea, gage, railroad...",3
5,"Mentally, Young Adults Are Suffering Most From...","Mentally, Young Adults Are Suffering Most From...",https://medium.com/the-partnered-pen/mentally-...,['Ryan Fan'],"['Society', 'Mental Health', 'Health', 'Nonfic...","[mental, young, adult, suffer, covid, “when, c...",4
...,...,...,...,...,...,...,...
9996,Adding a Border to the Image using NumPy,"In this article, we will learn how to add/draw...",https://medium.com/analytics-vidhya/adding-a-b...,['Sameeruddin Mohammed'],"['Numpy', 'Python Programming', 'Matrix Operat...","[articl, learn, adddraw, border, imag, use, op...",9995
9997,Connecting with Your Ancestors,Every person is a quotation from all their anc...,https://medium.com/grace-and-magic/connecting-...,['Katherine Grace'],"['Halloween', 'Self-awareness', 'Journaling', ...","[everi, person, quotat, ancestor, —, ralph, wa...",9996
9998,Learn how to create beautiful and insightful c...,The Pretty: Advanced plotting with Seaborn\n\n...,https://towardsdatascience.com/plotting-with-p...,['Fabian Bosler'],"['Data Visualization', 'Data Analysis', 'Data ...","[pretti, advanc, plot, seaborn, photo, pavel, ...",9997
9999,Five New Books to Get Your Hands On in 2020,Five New Books to Get Your Hands On in 2020\n\...,https://karintaglang.medium.com/five-new-books...,['Karin A. R. Taglang'],"['Books', 'Literature', 'Reading', 'Fiction', ...","[five, new, book, get, hand, 2020, ya, genr, i...",9998


In [66]:
df_new_part = df_part.loc[1:2]

Хранить будем в словаре, т. к. это быстрее всего.

In [84]:
def building_index(clean_data):
  index = {}
  for i, data in enumerate(clean_data):
    #print(i)
    for word in set(data):
      #print(word)
      if word not in index.keys():
        index[word] = []
        index[word].append(i)
      else:
        index[word].append(i)
  return index



Теперь у нас есть словарь, хранящий пары слово - номера документов, в которых слово встречается. Далее реализуем функцию поиска по индексу - если запрос состоит из одиночного слова, то достаточно найти его по ключу в словаре, если из большего количества - то пересечением списков поочередно(количество слов в запросе заведемо меньше чем количество ссылок - поэтому особых потерь времени не будет). 

In [88]:
#пересечение двух списков с помощью указателей
def find_intersection(word1, word2):
    l1 = index[word1]
    l2 = index[word2]
    out = []
    i1 = 0
    i2 = 0
    while (i1 < len(l1)) and (i2 < len(l2)):
        if l1[i1] > l2[i2]:
            i2 += 1
        elif l1[i1] < l2[i2]:
            i1 += 1
        else: #l1[i1] == l2[i2]
            out.append(l1[i1])
            i1 += 1
            i2 += 1
    return out

In [89]:
#пересечение списков количеством > 2
def multi_intersection(tokens):
  list_inter = []
  for i in range(len(tokens)- 1):
    list_inter = find_intersection(tokens[i], tokens[i+1])
  return list_inter

In [93]:
# получение списка релевантных документов
def get_list(request):
  tokens = analyze(request)
  if len(tokens) == 1:
    return index[tokens][:50] # первые 50 документов
  elif len(tokens) == 2:
    return find_intersection(tokens[0], tokens[1])[:50]
  else:
    return multi_intersection(tokens)[:50]

В качестве скора будем использовать косинусное расстояние для векторов запроса и отобранных ссылок:

In [95]:
from scipy.spatial.distance import cosine

In [ ]:
#функция для обработки запроса:
def request_clean(request):
  return analyze(request)

In [94]:
def cos(request, document):
  tfidf = TfidfVectorizer()
  req_vect = tfidf.fit_transform(request)
  doc_vect = tfidf.fit_transform(document)
  return cosine(req_vect, doc_vect)

К сожалению не хватает времени на отладку, я накидал примерный набор функций который может понадобиться.
